In [1]:
import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy


from scipy.optimize import curve_fit

In [2]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     

In [3]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        

In [4]:
energy_groups_2 = [0, 2 * 1E10]

In [5]:
class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)    

In [6]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [7]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [1]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    
              

In [2]:
class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()
            
    import random

In [3]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [4]:
class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()

In [5]:
import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [6]:
def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [7]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [8]:
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False

In [9]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [16]:
def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        

In [17]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [18]:
def process_one_particle_history(particle, universe, c_s, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
        for k in estimators:
            k.add_collision(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, c_s)
        
            
    return particle, sum_collisions

In [19]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [20]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])

In [21]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [22]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [23]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [24]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [25]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, c_s, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
    #print(estimators[1].collision_sum)
      #  assert(False)

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [26]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]/(sig_t * sum(weight_previos))
        estimator.collision_sum.append(0.)    

In [27]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))
            

In [28]:
cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]

In [29]:
def get_quarter_slab_estimators(radius):
    
    estimators = []
    
    x_5 = Plane(1, 0, 0, 10000)
    x_m5 = Plane(1, 0, 0, -10000)

    center_point = 0.1
    y_5 = Plane(0, 1, 0,  center_point)


    y_m5 = Plane(0, 1, 0, -center_point)


    z_5 = Plane(0, 0, 1, 10000)
    z_m5 = Plane(0, 0, 1, -10000)

    surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

    signs = [-1, +1, -1, +1, -1, +1]
    current_slub = Cell(surfaces, signs)

    estimators.append(Flux_estimator(current_slub))
    
    points_to_divide = [0.25, 0.5, 0.75]
    epsilon = 0.05
    
    for point in points_to_divide:
        
        x_5 = Plane(1, 0, 0, 10000)
        x_m5 = Plane(1, 0, 0, -10000)
        
        if point < 1:
            y_5 = Plane(0, 1, 0, point * radius + epsilon)
            
        else:
            y_5 = Plane(0, 1, 0, radius)
            
        y_m5 = Plane(0, 1, 0, point * radius - epsilon)


        z_5 = Plane(0, 0, 1, 10000)
        z_m5 = Plane(0, 0, 1, -10000)

        surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

        signs = [-1, +1, -1, +1, -1, +1]
        current_slub = Cell(surfaces, signs)
        
        estimators.append(Flux_estimator(current_slub))
        
    
    x_5 = Plane(1, 0, 0, 10000)
    x_m5 = Plane(1, 0, 0, -10000)
    y_5 = Plane(0, 1, 0, radius)
            
    y_m5 = Plane(0, 1, 0, radius - 0.02)


    z_5 = Plane(0, 0, 1, 10000)
    z_m5 = Plane(0, 0, 1, -10000)

    surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

    signs = [-1, +1, -1, +1, -1, +1]
    current_slub = Cell(surfaces, signs)
        
    estimators.append(Flux_estimator(current_slub))
        
    return estimators

In [30]:
estimators = get_quarter_slab_estimators(2.256751)

In [31]:
def save_flux_to_file(estimators, file_name):
    
    
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()   

In [44]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 2.256751)

y_m5 = Plane(0, 1, 0, -2.256751)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = [ "reflective", "reflective", "black", "black", "reflective", "reflective"]

slab.set_box_sizes(-1000, -2.849725, -1000, 1000, 2.849725, 1000 )
slab.set_zero_point(0., 0., 0.)

slab.set_boundaries_type(boundaries_type)

In [45]:
test_number_of_particles = 20
test_number_interations = 300
test_number_inactive = 20

In [46]:
print(len(estimators))

5


In [47]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(slab,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, estimators)

 num1  7
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
 keff_cycle , k_effective_exp, std_k_effective 0.9862713899989779   0.989060466415271  0.0027890764162931125
i == 23
 keff_cycle , k_effective_exp, std_k_effective 1.0089569448096811   0.9956926258800743  0.006824845901939599
i == 24
 keff_cycle , k_effective_exp, std_k_effective 1.010417870919292   0.9993739371398787  0.006069704554932918
i == 25
 keff_cycle , k_effective_exp, std_k_effective 0.9730621716982819   0.9941115840515593  0.007056709435469798
i == 26
 keff_cycle , k_effective_exp, std_k_effective 1.0083550321812733   0.9964854920731784  0.006231656120128845
i == 27
 keff_cycle , k_effective_exp, std_k_effective 1.0079905985991997   0.9981290787197529  0.005517211115503944
i == 28
 keff_cycle , k_effective_exp, std_k_effective 1.0036395112757848   0.9988178827892569  0.004827438753205002
i == 29

 keff_cycle , k_effective_exp, std_k_effective 0.9864869216591199   0.9959294689740453  0.0017277467428088824
i == 92
 keff_cycle , k_effective_exp, std_k_effective 1.0084303506827366   0.9961030923311104  0.0017124059648091572
i == 93
 keff_cycle , k_effective_exp, std_k_effective 1.017363881704751   0.9963943360211603  0.0017137150149136057
i == 94
 keff_cycle , k_effective_exp, std_k_effective 0.9969394307199873   0.9964017021657391  0.001690414127313926
i == 95
 keff_cycle , k_effective_exp, std_k_effective 1.0171491564090744   0.9966783348889836  0.001690510451750266
i == 96
 keff_cycle , k_effective_exp, std_k_effective 0.9864256022692387   0.9965434305124079  0.001673564710963871
i == 97
 keff_cycle , k_effective_exp, std_k_effective 0.9924230775886527   0.996489919435476  0.00165255370133408
i == 98
 keff_cycle , k_effective_exp, std_k_effective 0.9903555226141211   0.9964112733223818  0.001633124307869551
i == 99
 keff_cycle , k_effective_exp, std_k_effective 1.015554059382217

 keff_cycle , k_effective_exp, std_k_effective 0.9797202088257918   0.9978565766515962  0.0012229724650293385
i == 162
 keff_cycle , k_effective_exp, std_k_effective 1.0310670160531517   0.9980904529854101  0.0012366463248207858
i == 163
 keff_cycle , k_effective_exp, std_k_effective 1.021064807052218   0.9982511128040591  0.001238433276722052
i == 164
 keff_cycle , k_effective_exp, std_k_effective 1.0058387471112213   0.9983038047089701  0.001230931270624414
i == 165
 keff_cycle , k_effective_exp, std_k_effective 1.0243448383387788   0.998483398044348  0.001235534848544704
i == 166
 keff_cycle , k_effective_exp, std_k_effective 1.0050435921957237   0.9985283308810013  0.001227865516975913
i == 167
 keff_cycle , k_effective_exp, std_k_effective 0.9989090491770144   0.9985309208013823  0.001219486834635812
i == 168
 keff_cycle , k_effective_exp, std_k_effective 1.0170839715935054   0.9986562792526804  0.0012176889112895288
i == 169
 keff_cycle , k_effective_exp, std_k_effective 1.003114

 keff_cycle , k_effective_exp, std_k_effective 1.0012042029840849   0.9992265988825044  0.0010022914094605904
i == 232
 keff_cycle , k_effective_exp, std_k_effective 0.9916670632066525   0.9991909406953542  0.0009981895260174102
i == 233
 keff_cycle , k_effective_exp, std_k_effective 1.0193122344425893   0.9992854068631816  0.0009979731880602547
i == 234
 keff_cycle , k_effective_exp, std_k_effective 1.0220670373658012   0.9993918630804837  0.0009989872180563568
i == 235
 keff_cycle , k_effective_exp, std_k_effective 0.9796219518361948   0.9992999100049289  0.0009985726499737973
i == 236
 keff_cycle , k_effective_exp, std_k_effective 1.0113944704931164   0.9993559033405224  0.0009955148141494725
i == 237
 keff_cycle , k_effective_exp, std_k_effective 1.0044046510881675   0.9993791694591752  0.0009911896682442789
i == 238
 keff_cycle , k_effective_exp, std_k_effective 1.0246610164736796   0.999495141234471  0.0009934249063257604
i == 239
 keff_cycle , k_effective_exp, std_k_effective 0.

In [48]:
difference = (k_effective_exp[-1] - 1.) * 100000
k_effective_std = get_std(k_effective)
standart_deviation = k_effective_std * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -19.89498234560294  with standart deviation [pcm]+- 87.14491651804441


In [49]:
file_name = "result_flux"
save_flux_to_file(estimators, file_name)

In [50]:
def get_normalise_flux(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
    
    zero_point = average_number_of_collision[0]/(0.1/0.05)
    
    result_flux = []
    for i in range(1, len(average_number_of_collision)):
        result_flux.append(average_number_of_collision[i] / zero_point)
        
    result_flux[-1] = result_flux[-1] /(0.02/0.05) *2.
    
    result_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[200:len(current_estimator.collision_sum) - 1]
        current_std = get_std(array_without_zeros)
        result_std.append(current_std)  
        
    return result_flux, result_std, zero_point
    
    

In [51]:
result_flux, result_std, zero_point = get_normalise_flux(estimators_result)

In [52]:
print(result_flux)

[0.9743034281884867, 0.8823459455437538, 0.7302989970126537, 0.4946220304012589]


In [53]:
print(result_std)

[0.0006984953798472252, 0.0004985927135343995, 0.00047857070194182145, 0.00040662364505486857, 0.00015664856364932606]


In [54]:
flux_slub = [0.9701734, 0.8810540, 0.7318131, 0.4902592]

In [55]:
for i in range(0, len(flux_slub)):
    print(str(((flux_slub[i] - result_flux[i])/flux_slub[i])* 100) + " %  " + str(result_std[i] * 100000))

-0.4257000025445693 %  69.84953798472252
-0.14663636323696172 %  49.85927135343995
0.20689749709950986 %  47.85707019418214
-0.8899028108516677 %  40.662364505486856


In [56]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [57]:
file_name = "output"
save_simulation_results_to_file(k_effective, file_name)

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy


from scipy.optimize import curve_fit


# In[2]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     


# In[3]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        


# In[4]:


energy_groups_2 = [0, 2 * 1E10]


# In[5]:


class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)    


# In[6]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[7]:


class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    


# In[8]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    
              


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[10]:


class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()


# In[11]:


import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[12]:


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[13]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[14]:


def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False


# In[15]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)


# In[16]:


def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        


# In[17]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[18]:


def process_one_particle_history(particle, universe, c_s, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
        for k in estimators:
            k.add_collision(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, c_s)
        
            
    return particle, sum_collisions


# In[19]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[20]:


def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])


# In[21]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[22]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


# In[23]:


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[24]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[25]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, c_s, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
    #print(estimators[1].collision_sum)
      #  assert(False)

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[26]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]/(sig_t * sum(weight_previos))
        estimator.collision_sum.append(0.)    


# In[27]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))
            


# In[28]:


cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]


# In[32]:


def get_quarter_slab_estimators(radius):
    
    estimators = []
    
    points_to_divide = [0., 0.25, 0.5, 0.75]
    epsilon = 0.2
    
    for point in points_to_divide:
        
        x_5 = Plane(1, 0, 0, 10000)
        x_m5 = Plane(1, 0, 0, -10000)
        
        if point < 1:
            y_5 = Plane(0, 1, 0, point * radius + epsilon)
            
        else:
            y_5 = Plane(0, 1, 0, radius)
            
        y_m5 = Plane(0, 1, 0, point * radius - epsilon)


        z_5 = Plane(0, 0, 1, 10000)
        z_m5 = Plane(0, 0, 1, -10000)

        surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

        signs = [-1, +1, -1, +1, -1, +1]
        current_slub = Cell(surfaces, signs)
        
        estimators.append(Flux_estimator(current_slub))
        
    
    x_5 = Plane(1, 0, 0, 10000)
    x_m5 = Plane(1, 0, 0, -10000)
    y_5 = Plane(0, 1, 0, radius)
            
    y_m5 = Plane(0, 1, 0, radius - 0.02)


    z_5 = Plane(0, 0, 1, 10000)
    z_m5 = Plane(0, 0, 1, -10000)

    surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

    signs = [-1, +1, -1, +1, -1, +1]
    current_slub = Cell(surfaces, signs)
        
    estimators.append(Flux_estimator(current_slub))
        
    return estimators


# In[33]:


estimators = get_quarter_slab_estimators(2.256751)


# In[54]:


def save_simulation_results_to_file(k_effective, k_effective_exp, k_effective_std, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + "  average k effective "+ str(k_effective_exp[i]) + "  std "+ 
                 '\n')
            
    
    f.close()


# In[55]:


def save_flux_to_file(estimators, file_name):
    
    
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()
            
    
    


# In[50]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 2.256751)

y_m5 = Plane(0, 1, 0, -2.256751)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = [ "reflective", "reflective", "black", "black", "reflective", "reflective"]

slab.set_box_sizes(-1000, -2.849725, -1000, 1000, 2.849725, 1000 )
slab.set_zero_point(0., 0., 0.)

slab.set_boundaries_type(boundaries_type)


# In[51]:


test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 20


# In[38]:


print(len(estimators))


# In[39]:


k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(slab,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, estimators)


# In[40]:


difference = (k_effective_exp[-1] - 1.) * 100000
k_effective_std = get_std(k_effective)
standart_deviation = k_effective_std * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[57]:


file_name = "result_flux"
save_flux_to_file(estimators, file_name)


# In[42]:


def get_normalise_flux(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[200:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
    
    zero_point = average_number_of_collision[0]
    
    result_flux = []
    for i in range(1, len(average_number_of_collision)):
        result_flux.append(average_number_of_collision[i] / zero_point)
        
    result_flux[-1] = result_flux[-1] /0.05
    
    result_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[200:len(current_estimator.collision_sum) - 1]
        current_std = get_std(array_without_zeros)
        result_std.append(current_std)  
        
    return result_flux, result_std, zero_point
    
    


# In[43]:


result_flux, result_std, zero_point = get_normalise_flux(estimators_result)


# In[44]:


print(result_flux)


# In[45]:


print(result_std)


# In[46]:


flux_slub = [0.9701734, 0.8810540, 0.7318131, 0.4902592]


# In[47]:


for i in range(0, len(flux_slub)):
    print(str(((flux_slub[i] - result_flux[i])/flux_slub[i])* 100) + " %  " + str(result_std[i] * 100000))


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output"
save_simulation_results_to_file(k_effective, file_name)




5
 num1  9
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
 keff_cycle , k_effective_exp, std_k_effective 0.9804554476664448   0.983713165985602  0.0032577183191571657
i == 23
 keff_cycle , k_effective_exp, std_k_effective 1.0195349242655005   0.9956537520789016  0.012087810904673993
i == 24
 keff_cycle , k_effective_exp, std_k_effective 1.0276689545478503   1.0036575526961387  0.011709757066398289
i == 25
 keff_cycle , k_effective_exp, std_k_effective 0.986091916915423   1.0001444255399954  0.009726926994054658
i == 26
 keff_cycle , k_effective_exp, std_k_effective 1.035028071969599   1.005958366611596  0.009842627526012733
i == 27
 keff_cycle , k_effective_exp, std_k_effective 0.916402780843924   0.9931647115019286  0.015260265212706388
i == 28
 keff_cycle , k_effective_exp, std_k_effective 0.9340780928858486   0.9857788841749187  0.015139591014159208
i == 29


 keff_cycle , k_effective_exp, std_k_effective 1.0358524288137168   0.9882825720524995  0.005563279653041981
i == 93
 keff_cycle , k_effective_exp, std_k_effective 0.9590404226566694   0.9878819946635156  0.0055011449109811095
i == 94
 keff_cycle , k_effective_exp, std_k_effective 1.0195742624029278   0.988310268551886  0.005443170571747481
i == 95
 keff_cycle , k_effective_exp, std_k_effective 1.0044061094171781   0.9885248797634231  0.00537439122338257
i == 96
 keff_cycle , k_effective_exp, std_k_effective 0.9135942153231186   0.9875389499681559  0.005394073709901645
i == 97
 keff_cycle , k_effective_exp, std_k_effective 1.0940763888891771   0.9889225530710264  0.005500422498709565
i == 98
 keff_cycle , k_effective_exp, std_k_effective 1.0407443096347855   0.9895869345654336  0.005469944262778214
i == 99
 keff_cycle , k_effective_exp, std_k_effective 1.0438675105948287   0.9902740304645398  0.005443796038633668
i == 100
 keff_cycle , k_effective_exp, std_k_effective 1.000805595213757

 keff_cycle , k_effective_exp, std_k_effective 0.9854390225499168   0.9946695467736982  0.0038133991690927566
i == 163
 keff_cycle , k_effective_exp, std_k_effective 0.9950311591194685   0.9946720755313609  0.003786638986902929
i == 164
 keff_cycle , k_effective_exp, std_k_effective 0.9815626442415359   0.9945810378140705  0.003761352812128792
i == 165
 keff_cycle , k_effective_exp, std_k_effective 1.0122355252761732   0.994702792900016  0.0037373061895485398
i == 166
 keff_cycle , k_effective_exp, std_k_effective 1.0284891706195467   0.99493420644604  0.003718827066257453
i == 167
 keff_cycle , k_effective_exp, std_k_effective 1.0257978146133027   0.9951441629641847  0.003699405039749186
i == 168
 keff_cycle , k_effective_exp, std_k_effective 1.0022195087759653   0.9951919693548048  0.0036746350281827178
i == 169
 keff_cycle , k_effective_exp, std_k_effective 1.003932128570899   0.9952506281414901  0.003650361063244908
i == 170
 keff_cycle , k_effective_exp, std_k_effective 1.00248078

 keff_cycle , k_effective_exp, std_k_effective 1.0419260805142512   0.9973126813637654  0.003119405758127883
i == 234
 keff_cycle , k_effective_exp, std_k_effective 1.0202884886659236   0.9974200449492895  0.003106650635967342
i == 235
 keff_cycle , k_effective_exp, std_k_effective 0.9525096551709503   0.9972111594154368  0.0030992147406009927
i == 236
 keff_cycle , k_effective_exp, std_k_effective 1.0257075144367025   0.9973430869849798  0.003087652908679856
i == 237
 keff_cycle , k_effective_exp, std_k_effective 0.9615442152308147   0.997178115225744  0.0030778156020550163
i == 238
 keff_cycle , k_effective_exp, std_k_effective 0.9254313018824321   0.9968490014030682  0.003081291450128873
i == 239
 keff_cycle , k_effective_exp, std_k_effective 0.9744768215399797   0.9967468453306341  0.0030688900936895165
i == 240
 keff_cycle , k_effective_exp, std_k_effective 0.9459682460105635   0.9965160335155427  0.0030636157614481727
i == 241
 keff_cycle , k_effective_exp, std_k_effective 1.0246

 keff_cycle , k_effective_exp, std_k_effective 1.0765743006568917   0.9973480819480172  0.0025956689346704168
i == 304
 keff_cycle , k_effective_exp, std_k_effective 1.0259036514602733   0.9974486297279899  0.0025884667104215477
i == 305
 keff_cycle , k_effective_exp, std_k_effective 0.9278272247025169   0.9972043440963215  0.0025909104026882743
i == 306
 keff_cycle , k_effective_exp, std_k_effective 0.9953701769512826   0.9971979309244857  0.002581843347165399
i == 307
 keff_cycle , k_effective_exp, std_k_effective 1.024787388308822   0.99729406143802  0.002574626919597026
i == 308
 keff_cycle , k_effective_exp, std_k_effective 0.9508881885641568   0.9971329299349857  0.0025707264481618407
i == 309
 keff_cycle , k_effective_exp, std_k_effective 0.9761359985018621   0.9970602761930026  0.0025628457899163617
i == 310
 keff_cycle , k_effective_exp, std_k_effective 0.9254710276271852   0.9968134167151894  0.0025658956256293395
i == 311
 keff_cycle , k_effective_exp, std_k_effective 0.9538

 keff_cycle , k_effective_exp, std_k_effective 0.9973378606847956   0.9965056290225875  0.002346111586740414
i == 374
 keff_cycle , k_effective_exp, std_k_effective 1.0059038506013696   0.9965321776711152  0.0023396253993820644
i == 375
 keff_cycle , k_effective_exp, std_k_effective 1.014332149526784   0.9965823184369059  0.0023335643404728992
i == 376
 keff_cycle , k_effective_exp, std_k_effective 1.0453019179792968   0.9967191712446093  0.002331020891990123
i == 377
 keff_cycle , k_effective_exp, std_k_effective 1.0305132847848477   0.9968138326270749  0.0023264089307182454
i == 378
 keff_cycle , k_effective_exp, std_k_effective 0.9132829185458446   0.9965805060514291  0.002331605491861311
i == 379
 keff_cycle , k_effective_exp, std_k_effective 1.0272382336408619   0.996665903621316  0.0023266694332965973
i == 380
 keff_cycle , k_effective_exp, std_k_effective 0.9974223805373703   0.9966680049460829  0.0023201984128285846
i == 381
 keff_cycle , k_effective_exp, std_k_effective 0.9774

 keff_cycle , k_effective_exp, std_k_effective 1.0554475506939314   0.9963664006000548  0.002130327505702293
i == 444
 keff_cycle , k_effective_exp, std_k_effective 1.003908484407498   0.9963841885335629  0.0021253716472455766
i == 445
 keff_cycle , k_effective_exp, std_k_effective 0.9636050647586102   0.9963070611835042  0.0021217671487341765
i == 446
 keff_cycle , k_effective_exp, std_k_effective 0.9994317815318922   0.9963143962077962  0.0021167933236003527
i == 447
 keff_cycle , k_effective_exp, std_k_effective 0.9741252203499157   0.9962624309247565  0.002112469395374643
i == 448
 keff_cycle , k_effective_exp, std_k_effective 0.9528594767332195   0.9961610221532811  0.002109966292543708
i == 449
 keff_cycle , k_effective_exp, std_k_effective 1.042533931089129   0.9962691175121058  0.0021078157686235588
i == 450
 keff_cycle , k_effective_exp, std_k_effective 0.9807335099495547   0.996232988192193  0.0021032184979348253
i == 451
 keff_cycle , k_effective_exp, std_k_effective 0.94917